# Setup DMA Memory Region

**IMPORTANT:** This notebook modifies system configuration and requires a reboot to take effect.

The default DMA region is only 2 MB, which limits capture to ~67ms at 15.625 MS/s.

We need to increase it to **128 MB** to support 2+ second captures.

| Size | Hex Value | Capture Duration @ 15.625 MS/s |
|------|-----------|--------------------------------|
| 2 MB | 0x200000 | 67 ms (default) |
| 64 MB | 0x4000000 | 2.1 seconds |
| 128 MB | 0x8000000 | 4.2 seconds |
| 256 MB | 0x10000000 | 8.4 seconds |

## Step 1: Check Current Configuration

In [ ]:
import subprocess
import os

# Get the FPGA model
result = subprocess.run(['monitor', '-f'], capture_output=True, text=True)
fpga_model = result.stdout.strip()
print(f"FPGA Model: {fpga_model}")

# Path to device tree source
dts_path = f"/opt/redpitaya/dts/{fpga_model}/dtraw.dts"
print(f"Device tree path: {dts_path}")

# Check if file exists
if os.path.exists(dts_path):
    print("Device tree file found!")
else:
    print(f"WARNING: File not found at {dts_path}")

In [ ]:
# Read and display the current buffer configuration
with open(dts_path, 'r') as f:
    content = f.read()

# Find the buffer section
import re
buffer_match = re.search(r'buffer@[0-9a-fA-F]+\s*\{[^}]+\}', content)
if buffer_match:
    print("Current buffer configuration:")
    print(buffer_match.group())
    
    # Extract current size
    reg_match = re.search(r'reg\s*=\s*<0x([0-9a-fA-F]+)\s+0x([0-9a-fA-F]+)>', buffer_match.group())
    if reg_match:
        start_addr = int(reg_match.group(1), 16)
        current_size = int(reg_match.group(2), 16)
        print(f"\nStart address: 0x{start_addr:X}")
        print(f"Current size: 0x{current_size:X} ({current_size / 1024 / 1024:.1f} MB)")
        print(f"Max samples: {current_size // 2:,}")
        print(f"Max duration @ 15.625 MS/s: {current_size // 2 / 15.625e6 * 1000:.1f} ms")
else:
    print("Could not find buffer configuration in device tree")

## Step 2: Modify DMA Size

**Run the cell below to increase DMA to 128 MB**

In [ ]:
# New size: 128 MB = 0x8000000
NEW_SIZE_HEX = "0x8000000"  # 128 MB - supports ~4.2 seconds of capture
NEW_SIZE_BYTES = 0x8000000

print(f"New DMA size: {NEW_SIZE_HEX} ({NEW_SIZE_BYTES / 1024 / 1024:.0f} MB)")
print(f"Max samples: {NEW_SIZE_BYTES // 2:,}")
print(f"Max duration @ 15.625 MS/s: {NEW_SIZE_BYTES // 2 / 15.625e6:.2f} seconds")
print("\n" + "="*50)
print("Proceed to next cell to apply changes")

In [ ]:
# Enable read-write mode on filesystem
print("Enabling read-write mode...")
result = subprocess.run(['rw'], capture_output=True, text=True, shell=True)
print("Done" if result.returncode == 0 else f"Note: {result.stderr}")

# Backup original file
backup_path = dts_path + ".backup"
if not os.path.exists(backup_path):
    import shutil
    shutil.copy(dts_path, backup_path)
    print(f"Backup created: {backup_path}")
else:
    print(f"Backup already exists: {backup_path}")

In [ ]:
# Modify the device tree
import re

with open(dts_path, 'r') as f:
    content = f.read()

# Replace the buffer size (keep start address, change size)
# Match pattern: reg = <0xSTART 0xSIZE>
new_content = re.sub(
    r'(buffer@[0-9a-fA-F]+\s*\{[^}]*reg\s*=\s*<0x[0-9a-fA-F]+)\s+0x[0-9a-fA-F]+(>[^}]*\})',
    rf'\1 {NEW_SIZE_HEX}\2',
    content
)

if new_content != content:
    with open(dts_path, 'w') as f:
        f.write(new_content)
    print("Device tree modified successfully!")
    
    # Verify the change
    buffer_match = re.search(r'buffer@[0-9a-fA-F]+\s*\{[^}]+\}', new_content)
    if buffer_match:
        print("\nNew configuration:")
        print(buffer_match.group())
else:
    print("No changes made (pattern not found or already at target size)")

## Step 3: Compile Device Tree and Reboot

**WARNING: This will reboot the Red Pitaya!**

In [ ]:
# Compile the device tree
dts_dir = os.path.dirname(dts_path)
print(f"Compiling device tree in {dts_dir}...")

result = subprocess.run(
    ['dtc', '-I', 'dts', '-O', 'dtb', './dtraw.dts', '-o', 'devicetree.dtb'],
    cwd=dts_dir,
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print("Device tree compiled successfully!")
    print("\n" + "="*50)
    print("Ready to reboot. Run the next cell to reboot.")
else:
    print(f"Compilation failed: {result.stderr}")

In [ ]:
# REBOOT - uncomment and run to reboot
print("Rebooting in 3 seconds...")
import time
time.sleep(3)
os.system('reboot')

## After Reboot

After the Red Pitaya reboots (~30-60 seconds), re-open this notebook and run the cell below to verify the new DMA size.

In [ ]:
# Verify new DMA size
from rp_overlay import overlay
import rp

fpga = overlay()
rp.rp_Init()

memoryRegion = rp.rp_AcqAxiGetMemoryRegion()
g_adc_axi_start = memoryRegion[1]
g_adc_axi_size = memoryRegion[2]

print(f"DMA Memory Region:")
print(f"  Start address: 0x{g_adc_axi_start:08X}")
print(f"  Size: {g_adc_axi_size:,} bytes ({g_adc_axi_size / 1024 / 1024:.1f} MB)")
print(f"  Max samples: {g_adc_axi_size // 2:,}")
print(f"  Max duration @ 15.625 MS/s: {g_adc_axi_size // 2 / 15.625e6:.2f} seconds")

rp.rp_Release()